In [ ]:
'''
License - Any one can use the code or specific part for any purpose with acknowledgement of work of original Authors of this code. 

Trust Algorithm is parallerlized on GPU for graph Matching. Achieved speeds are between 150-300, on K140. Speedups might vary. 

The graph data - the data generated in this program is an artificially generated graph which is bidirectonal in nature , however real practical graphs can be more complex than that. 
However, the code serves as a good example of how to accelerate an heuristic beam search algorithm on GPU using Cuda and python. 

Authors of the code - 
1. Xuan Zhang
2. Dushyant Singh Udawat
3. Leo franco Soto
'''


# Importing all the libraries that will be required.
import numpy as np
from typing import List, Dict, Tuple
import heapq
import random
import time
import cupy as cp
from numba import cuda
import numba

p = ""
q = ""

# return the normalized distance between two words
def word_distance(p : str, q : str) -> float:
    # place holder, now we use simple way to calculate the distance between two string
    n = len(p)
    m = len(q)
    #print(n, m)
    d = np.zeros([n + 1, m + 1], dtype=int)
    for j in range(m + 1):
        d[0][j] = j
    for i in range(1, n + 1):
        d[i][0] = i
        for j in range(1, m + 1):
            d[i][j] = min(d[i][j - 1], d[i - 1][j]) + 1
            d[i][j] = min(d[i][j], d[i - 1][j - 1] + (p[i - 1] != q[j - 1]))

#    print(d)
    return d[n][m] / max(n, m)

#p = input()
#q = input()

#print(word_distance(p, q))

lack = 0.
eps = 1e-7


# this is normal max-matching algorithm - max matching is n-p hard. We use quadratic solver. 
def max_matching_algo(w : np.array, n: int) -> float:
 #   print("matching....")
    lx = np.zeros([n + 2], dtype = float)
    ly = np.zeros([n + 2], dtype = float)
    visited_x = np.zeros([n + 2], dtype=int)
    visited_y = np.zeros([n + 2], dtype=int)
    link_y = np.zeros([n + 2], dtype=int)
    global lack

    def find(x: int) -> bool:
        global lack
        visited_x[x] = True
        for y in range(1, n + 1):
            if not visited_y[y]:
                tmp = lx[x] + ly[y] - w[x][y]
                if tmp < eps:
                    visited_y[y] = True
                    if link_y[y] == 0 or find(link_y[y]):
                        link_y[y] = x
                        return True
                else:
                    lack = min(lack, tmp)
        return False
    for i in range(1, n + 1):
        for j in range(1, n + 1):
            lx[i] = max(lx[i], w[i][j]) # init

    for x in range(1, n + 1):
        while True:
            visited_x = np.zeros([n + 2], dtype=int)
            visited_y = np.zeros([n + 2], dtype=int)
            lack = 100000
            if find(x): break
            for i in range(1, n + 1):
                if visited_x[i]:
                    lx[i] -= lack
                if visited_y[i]:
                    ly[i] += lack
    total_value = 0.

    for y in range(1, n + 1):
        total_value += w[link_y[y]][y]

  #  print("matching Complete!")
    return total_value


# A recursive Max matching algorithm function is not cuda implementable , so here is a non recursive version of the same. 
def Non_recursive_max_matching_algo(w : np.array, n: int) -> float:
 #   print("matching....")
    lx = np.zeros([n + 2], dtype = float)
    ly = np.zeros([n + 2], dtype = float)
    visited_x = np.zeros([n + 2], dtype=int)
    visited_y = np.zeros([n + 2], dtype=int)
    link_y = np.zeros([n + 2], dtype=int)

    for i in range(1, n + 1):
        for j in range(1, n + 1):
            lx[i] = max(lx[i], w[i][j]) # init

    for x in range(1, n + 1):
      while True:
        lack_GPU = 100000
        visited_x = np.zeros([n + 2], dtype=int)
        visited_y = np.zeros([n + 2], dtype=int)

        pos = 1
        stack_x = np.zeros(n, dtype = int)
        stack_y = np.zeros(n, dtype = int)
        stack_x[1] = x
        stack_y[1] = 0
        Label = False
        while pos > 0:
          cur_x = stack_x[pos]
          cur_y = stack_y[pos]
          Recur_label = False
          if Label:
            link_y[cur_y] = cur_x
            pos -= 1
            continue
          visited_x[cur_x] = True
          for y in range(cur_y + 1, n + 1):
            if not visited_y[y]:
              tmp_GPU = lx[cur_x] + ly[y] - w[cur_x][y]
              if tmp_GPU < eps:
                  visited_y[y] = True
                  if link_y[y] == 0:
                    link_y[y] = cur_x
                    pos -=1
                    Label = True
                  else:
                    stack_y[pos] = y
                    pos += 1
                    stack_x[pos] = link_y[y]
                    stack_y[pos] = 0
                    Recur_label = True
                  break
              else:
                  lack_GPU = min(lack_GPU, tmp_GPU)
          if Recur_label == False and Label == False: pos -= 1

        if Label: break

        for i in range(1, n + 1):
            if visited_x[i]:
                lx[i] -= lack_GPU
            if visited_y[i]:
                ly[i] += lack_GPU
    total_value = 0.

    for y in range(1, n + 1):
        total_value += w[link_y[y]][y]

  #  print("matching Complete!")
    return total_value


  

In [ ]:
# here we are calculating hop scores for each pairs of graph nodes. When triggering this function in GPU we will call as many threads as there are node pairs in the graph.
@cuda.jit
def GPU_calculate_hop_score_i_j(gpu_data_n, gpu_word_distance, gpu_data_graph_adj_matrix, gpu_template_graph_adj_matrix, gpu_data_node_name_list, gpu_template_node_name_list, gpu_one_hop_score_table):
    #   print("matching....")
    id_x, id_y = cuda.grid(2)
    id_x += 1
    id_y += 1

    if id_x > gpu_data_n:
      return
    n = gpu_data_graph_adj_matrix[id_x][0]
    m = gpu_template_graph_adj_matrix[id_y][0]
    #print(n, m)
    max_nm = max(n, m)
    if max_nm == 0:
      gpu_one_hop_score_table[id_x][id_y] = 0.5 * gpu_word_distance[id_x][id_y]
    else:
      cnt = 0.
      cnt_n = 0
      # w = cuda.local.array(shape = (52, 52), dtype=numba.float64)
      # for x in range(1, n + 1):
      #  p = gpu_data_graph_adj_matrix[id_x][x]
      #  for y in range(1, m + 1):
      #    q = gpu_template_graph_adj_matrix[id_y][y]
      #    w[x][y] = gpu_word_distance[p][q]

      lx = cuda.local.array(80, dtype = numba.float64)
      ly = cuda.local.array(80, dtype = numba.float64)
      visited_x = cuda.local.array(80, dtype=numba.boolean)
      visited_y = cuda.local.array(80, dtype=numba.boolean)
      link_y = cuda.local.array(80, dtype=numba.int64)
      for i in range(1, max_nm + 1):
        lx[i] = ly[i] = 0
      for i in range(1, max_nm + 1):
          for j in range(1, max_nm + 1):
              lx[i] = max(lx[i], gpu_word_distance[gpu_data_graph_adj_matrix[id_x][i]][gpu_template_graph_adj_matrix[id_y][j]]) # init

      for x in range(1, max_nm + 1):
        while True:
          lack_GPU = 100000
          for i in range(1, n + 1):
            visited_x[i] = 0
            visited_y[i] = 0

          pos = 1
          stack_x = cuda.local.array(80, dtype = numba.int64)
          stack_y = cuda.local.array(80, dtype = numba.int64)
          stack_x[1] = x
          stack_y[1] = 0
          Label = False
          while pos > 0:
            cur_x = stack_x[pos]
            cur_y = stack_y[pos]
            Recur_label = False
            if Label:
              link_y[cur_y] = cur_x
              pos -= 1
              continue
            visited_x[cur_x] = True
            for y in range(cur_y + 1, max_nm + 1):
              if not visited_y[y]:
                tmp_GPU = lx[cur_x] + ly[y] - gpu_word_distance[gpu_data_graph_adj_matrix[id_x][cur_x]][gpu_template_graph_adj_matrix[id_y][y]]
                if tmp_GPU < eps:
                    visited_y[y] = True
                    if link_y[y] == 0:
                      link_y[y] = cur_x
                      pos -=1
                      Label = True
                    else:
                      stack_y[pos] = y
                      pos += 1
                      stack_x[pos] = link_y[y]
                      stack_y[pos] = 0
                      Recur_label = True
                    break
                else:
                    lack_GPU = min(lack_GPU, tmp_GPU)
            if Recur_label == False and Label == False: pos -= 1

          if Label: break

          for i in range(1, max_nm + 1):
              if visited_x[i]:
                  lx[i] -= lack_GPU
              if visited_y[i]:
                  ly[i] += lack_GPU

      max_matching_value = 0.
      for y in range(1, max_nm + 1):
        max_matching_value += gpu_word_distance[gpu_data_graph_adj_matrix[id_x][link_y[y]]][gpu_template_graph_adj_matrix[id_y][y]]

      gpu_one_hop_score_table[id_x][id_y] = 0.5 * gpu_word_distance[id_x][id_y] + 0.5 * max_matching_value/max_nm

# these Node/Edge class are for general Node/Edge, not for specific nodes


#A node have a name on it and a weight on it, now we consider the weight as a part of the name.

# Specifications of the search processes. Changing them will lead to change in how the matching algorithm works. 
DETAIL_RESULT = False
HEAP_SIZE = 1000
SEARCH_RANGE = 1000
HASHING_NUM =  2**16 + 1
HASHING_TOTAL =  2**32 + 1
ADJ_MAX = 50
#print(HASHING_TOTAL, HASHING_NUM)
Hashing_A = 2
Hashing_B = 3
TEMPLATE_GRAPH_MAX_SIZE = 10
cuda.detect()


# this is the Node class of the graph. It contains a name, and value, it is identified by the name. attr stores combination of name and value. 
class Node:
    name = ""
    value = 0
    __attr = ""

    def __init__(self, name="", value=0):
        self.name = name
        self.value = value
        self.__attr = self.name + "|" + str(self.value)

    def name_distance(self, other) -> float:
        return word_distance(self.name, other.name)

    def attr(self):
        return self.__attr

    def change_name(self):
        # todo
        pass


# An edge contain the "name" of two node, the name for itself and the value/capacity/weight on the edge
class Edge:
    def __init__(self, name="", from_node_name: str = "", to_node_name: str = "", value=0):
        self.name = name
        self.from_node_name = from_node_name
        self.to_node_name = to_node_name
        self.value = value
        self.__attr = name + '|' + self.from_node_name + '|' + self.to_node_name + '|' + str(self.value)

    def attr(self):
        return self.__attr

# This is a update function which will set the values of all the variablles correctly to make everything ready for the next iteration of GPU_trust_searching_matrix_type
@cuda.jit
def GPU_search_update_history_position(cur_level, p, tmp_value, tmp_history, searching_history, searching_value, nn):
  i = cuda.grid(1)
  searching_value[cur_level][nn - i - 1] = tmp_value[p[i]]
  if tmp_history[p[i]][1] != 0 and tmp_history[p[i]][1] != 0:
    #print(type(searching_history))
    searching_history[cur_level][nn - i - 1][0] = tmp_history[p[i]][0]
    searching_history[cur_level][nn - i - 1][1] = tmp_history[p[i]][1]
    searching_history[cur_level][nn - i - 1][2] = tmp_history[p[i]][2]

# This is the function that finds the best matching at each of the steps in Beam search algorithm
@cuda.jit
def GPU__trust_searching_matrix_type(DATA_NODE_SIZE : int, TEMPLATE_NODE_SIZE : int, current_lvl, GPU_one_hop_score_table, GPU_template_graph_adj_matrix, GPU_data_graph_adj_matrix,searching_history, searching_value, new_history, new_value):
  matching_info = cuda.local.array(shape=50, dtype=numba.int64)
  adj_check = cuda.local.array(shape=50, dtype=numba.int64)
  check_index = cuda.grid(1)
  searching_index = cuda.grid(1)
  tmp : int = 0
  for p in range(current_lvl, 0, -1):
    check_index, x, y = searching_history[p][check_index]
    if x == 0 or y == 0:
      return
    matching_info[y] = x
  for tt in range(1, TEMPLATE_NODE_SIZE + 1):
    if matching_info[tt] != 0:
      for i_index in range(1, GPU_data_graph_adj_matrix[matching_info[tt]][0] + 1):
        i = GPU_data_graph_adj_matrix[matching_info[tt]][i_index]
        already_matched = False
        for check_y in range(1, TEMPLATE_NODE_SIZE + 1):
          if matching_info[check_y] == i:
            already_matched = True
            break
          adj_check[check_y] = 0
        if already_matched: continue
        check_sum = 0
        for check_x_index in range(1, GPU_data_graph_adj_matrix[i][0] + 1):
          check_x = GPU_data_graph_adj_matrix[i][check_x_index] 
          for check_y in range(0, TEMPLATE_NODE_SIZE + 1):
            if check_x == matching_info[check_y]:
              adj_check[check_y] = 1
              check_sum += 1
        for j in range(1, TEMPLATE_NODE_SIZE + 1):
          if matching_info[j] != 0: continue
          check_flag = True
          check_sum_tmp = check_sum
          for check_y_jndex in range(1, GPU_template_graph_adj_matrix[j][0] + 1):
            check_y = GPU_template_graph_adj_matrix[j][check_y_jndex]
            if matching_info[check_y] == 0:
              continue
            if adj_check[check_y] == 0:
              check_flag = False
              break
            else:
              check_sum_tmp -= 1
          if check_flag and check_sum_tmp == 0:
              # matching_info[j] = i
              # if self.check_hashing_matrix(matching_info, template_graph.n):
              #     print("YES")
              tmp_value = searching_value[current_lvl][searching_index] + GPU_one_hop_score_table[i][j]
              s = int(searching_index * ADJ_MAX * TEMPLATE_NODE_SIZE + tmp)
              new_value[s] = tmp_value
              new_history[s][0] = int(searching_index)
              new_history[s][1] = int(i)
              new_history[s][2] = int(j)
              tmp += 1
              if tmp >= 50 * TEMPLATE_NODE_SIZE:
                return
              #self.updating_new_instance_matrix_base(new_value, current_lvl + 1, searching_index,i, j)

# def name_distance(self, other) -> float:
#    return word_distance(self.name, other.name)

#GraphNode contains the information of a "node" and its relationship in the graph
class GraphNode:
    def __init__(self, node: Node, node_index=-1):
        self.node_index = node_index
        self.node_info = node
        self.adjc_edge: Dict[str, GraphEdge] = {}
        self.adjc_node: Dict[str, GraphNode] = {}

    def similarity_score(self, other) -> float:
        return Node.name_distance(self.node_info, other.node_info)

    def one_hop_distance(self, other) -> float:
        n = len(self.adjc_edge)
        m = len(other.adjc_edge)
        max_nm = max(n, m)
        if max_nm == 0:
            return 0.5 * self.similarity_score(other)
        cnt = 0.
        cnt_n = 0
        t = np.zeros([max_nm + 1, max_nm + 1], dtype=float)
        for i in self.adjc_edge:
            edge_i = self.adjc_edge.get(i)
            cnt_n += 1
            if edge_i.from_node == self:
                node_i = edge_i.to_node
            else:
                node_i = edge_i.from_node
            cnt_m = 0
            for j in other.adjc_edge:
                cnt_m += 1
                edge_j = other.adjc_edge.get(j)
                if edge_j.from_node == other:
                    node_j = edge_j.to_node
                else:
                    node_j = edge_j.from_node

                t[cnt_n][cnt_m] = GraphNode.similarity_score(node_i, node_j) #+ 0.5 * GraphEdge.name_distance(edge_i, edge_j)
        max_matching_value = max_matching_algo(t, max_nm) # get the max bipartite matching
        return 0.5 * self.similarity_score(other) + 0.5 * max_matching_value/max_nm



# GraphEdge connects two nodes in the graph, with name and value on the edge.
class GraphEdge:
    def __init__(self, from_node: GraphNode, to_node: GraphNode, name: str = "", value: int = 0):
        self.name = name
        self.from_node = from_node
        self.to_node = to_node
        self.value = value
        self.connect_num = (from_node.node_index, to_node.node_index)
        #  self.is_directed = is_directed

    def name_distance(self, other) -> float:
        return word_distance(self.name, other.name)

    def distance(self, other) -> float:
        return max(0.5 * GraphNode.similarity_score(self.from_node, other.from_node) + 0.5 * GraphNode.similarity_score(self.to_node, other.to_node),
                   0.5 * GraphNode.similarity_score(self.from_node, other.to_node) + 0.5 * GraphNode.similarity_score(self.to_node, other.from_node))


# for simplicity we assume it's a totally undirected graph for now
class Graph:
    # n, m are node_size, edge_size
    def __init__(self, n : int, m : int, node_list: List[Node], edge_list: List[Edge]):
        # here the node in the edge_list is corresponding to the node in the node_list
        self.n = 0
        self.m = m
        self.hashing_table = set()
        self.node_list : Dict[str, GraphNode] = {}
        self.heap_size = 0
        self.H: List[(int, int)] = []
        self.heap_info_list = [{}]
        # adj_matrix and indexing is 1-based
        self.adj_matrix = np.zeros([n + 2, ADJ_MAX + 2], dtype=int)

        self.word_distance_table = np.zeros((n + 2, ADJ_MAX + 2), dtype= float)
        self.searching_history = np.zeros([0], dtype= int)
        self.searching_value = np.zeros([0], dtype= float)

        self.node_name_list = ["-1"]
        self.one_hop_score_table_old = None
        self.one_hop_score_table_CPU = None
        self.one_hop_score_table = None

        # build_node
        for node in node_list:
            self.node_name_list += [node.name]
            if self.node_list.get(node.name) is None:
                self.n += 1
                self.node_list[node.name] = GraphNode(node, self.n)
        # build_edge
        self.edge_list : Dict[str, GraphEdge] = {}
        for edge in edge_list:
            self.build_edge(edge)

        self.one_hope_score_dict: Dict[str, float] = {}

    def build_edge(self, edge: Edge):
        if self.edge_list.get(edge.attr()) is not None:
            return

        from_node_name = edge.from_node_name
        to_node_name = edge.to_node_name

        # find GraphNode in the Graph Node list, if there is none, create a new Graph Node for it.
        p = self.node_list.get(from_node_name)
        if p is None:
            print("new_node_found, Error!")
            self.n += 1
            p = GraphNode(Node(from_node_name), self.n)
            self.node_list[from_node_name] = p
        q = self.node_list.get(to_node_name)
        if q is None:
            print("new_node_found, Error!")
            self.n += 1
            q = GraphNode(Node(to_node_name), self.n)
            self.node_list[to_node_name] = q

        # create an GraphEdge and add it into the edge list
        adding_edge = GraphEdge(p, q, edge.name, edge.value)
        self.edge_list[edge.attr()] = adding_edge

        # adding the adjacency edge to its two GraphNodes
        # here we first consider no duplicate edges
        p.adjc_edge[edge.attr()] = adding_edge
        p.adjc_node[q.node_info.name] = q
        q.adjc_edge[edge.attr()] = adding_edge
        q.adjc_node[p.node_info.name] = p


        # create adj_matrix
        self.adj_matrix[p.node_index][0] += 1
        self.adj_matrix[q.node_index][0] += 1
        self.adj_matrix[p.node_index][self.adj_matrix[p.node_index][0]] = q.node_index
        self.adj_matrix[q.node_index][self.adj_matrix[q.node_index][0]] = p.node_index

    # building an array of distances on class for easy access
    def build_word_distance(self, template_graph):
      for i in range(1, self.n+1):
        for j in range(1, template_graph.n+1):
          self.word_distance_table[i][j] = word_distance(self.node_name_list[i],template_graph.node_name_list[j])

    # building one_hop scores for each pair of nodes and saving for easier access.
    def GPU_build_node_based_one_hop_score(self, template_graph):
        self.one_hop_score_table = np.zeros([self.n + 2, template_graph.n + 2], dtype=float)
        data_size = self.n 
        tmp_size = template_graph.n
        num_i = 0
        gpu_word_distance = cuda.to_device(self.word_distance_table)
        gpu_template_graph_adj_matrix = cp.array(template_graph.adj_matrix)
        gpu_data_graph_adj_matrix = cp.array(self.adj_matrix)
        gpu_data_node_name_list = cuda.to_device(self.node_name_list)
        gpu_template_node_name_list = cuda.to_device(template_graph.node_name_list)
        gpu_one_hop_score_table = cp.array(self.one_hop_score_table)
        GPU_calculate_hop_score_i_j[(int(self.n/8)+1,template_graph.n),(8,1)](self.n, gpu_word_distance, gpu_data_graph_adj_matrix, gpu_template_graph_adj_matrix, gpu_data_node_name_list, gpu_template_node_name_list, gpu_one_hop_score_table)

        self.one_hop_score_table = cp.asnumpy(gpu_one_hop_score_table)

    # 
    def OLD_build_node_based_one_hop_score(self, template_graph):      
        self.one_hop_score_table_old = np.zeros([self.n + 2, template_graph.n + 1], dtype=float)
        data_size = len(self.node_list)
        tmp_size = len(template_graph.node_list)
        num_i = 0
      #  print(self.node_name_list)
      #  for i in self.node_list:
      #    print(self.node_list[i].node_index, i)
        for i in self.node_list:
            node_i = self.node_list.get(i)
            #print(i, '.....')
            for j in template_graph.node_list:
                node_j = template_graph.node_list.get(j)
                tmp_value = GraphNode.one_hop_distance(node_i, node_j)
    
                # print(node_i.node_index, node_j.node_index, tmp_value)

                # self.one_hope_score_dict[node_i.node_info.name + '|' + node_j.node_info.name] = tmp_value

                self.one_hop_score_table_old[node_i.node_index][node_j.node_index] = tmp_value

    def CPU_build_node_based_one_hop_score(self, template_graph):
        self.one_hop_score_table = np.zeros([self.n + 2, template_graph.n + 2], dtype=float)
        data_size = len(self.node_list)
        tmp_size = len(template_graph.node_list)
        num_i = 0
        for i in range(1, self.n + 1):
        #  print(self.node_name_list[i],"....")
          for j in range(1, template_graph.n + 1):
            n = self.adj_matrix[i][0]
            m = template_graph.adj_matrix[j][0]
            #print(n, m)
            max_nm = max(n, m)
            if max_nm == 0:
              self.one_hop_score_table[i][j] = 0.5 * self.word_distance_table[i][j]
            else:
              cnt = 0.
              cnt_n = 0
              t = np.zeros([max_nm + 1, max_nm + 1], dtype=float)
              for x in range(1, n + 1):
                p = self.adj_matrix[i][x]
                for y in range(1, m + 1):
                  q = template_graph.adj_matrix[j][y]
                  t[x][y] = self.word_distance_table[p][q]
              max_matching_value = max_matching_algo(t, max_nm)

              self.one_hop_score_table_CPU[i][j] = 0.5 * word_distance(self.node_name_list[i], template_graph.node_name_list[j]) + 0.5 * max_matching_value/max_nm

    def simple_hashing(self,fitting_detail):
        tmp = 0.
        for j in fitting_detail:
            i = fitting_detail.get(j)
            tmp = tmp + float(i) * float(j)
        if tmp in self.hashing_table:
            return False
        else:
            self.hashing_table.add(tmp)
            return True

    def search_clear_start(self):
        self.H = []
        self.hashing_table = set()
        self.heap_info_list = [{}]
        self.heap_size = 0

    #####################

    # BELOW USING THE MATRIX RATHER THAN THE DICTIONARY, HASHING STUFF HASN'T CHANGED YET

    ###################
    def check_hashing_matrix(self, matching_info, n):
        t = 1
        total = 0
        for i in range(1, n + 1):
            total = (total + t * matching_info[i]) % HASHING_TOTAL
     #       print(total)
            t = (t * HASHING_NUM) % HASHING_TOTAL
    #    print(total)
        if total in self.hashing_table:
    #        print(total)
            return False
        else:
      #      print(total)
            self.hashing_table.add(total)
            return True


      
    def GPU_search_update_results(self, cur_level, tmp_value, tmp_history, searching_history, searching_value):
      nn = cp.size(tmp_value)
      p = cp.argsort(tmp_value)[max(nn - SEARCH_RANGE, 0):nn]
      nn = cp.size(p)
      GPU_search_update_history_position[nn,1](cur_level, p, tmp_value, tmp_history, searching_history, searching_value, nn)
      
    def GPU_TRUST_clear_start_matrix(self, template_graph_size):
        self.hashing_table = set()
        self.searching_history = np.zeros([template_graph_size + 1, SEARCH_RANGE, 3], dtype=int)
        self.searching_value = np.zeros([template_graph_size + 1, SEARCH_RANGE], dtype=float)

    ### THIS IS THE MAIN GPU SEARCHING FUNCTION
    def GPU_search_best_fitting_graph_matrix_type(self, template_graph):
        self.GPU_TRUST_clear_start_matrix(template_graph.n)
        # searching_history: [lvl, instance num, 3]
        #                    where each instance = [father instance index, data_node_index, template_node_index]
        # searching_history: Corresponding value of "newest instance"
        # THE FATHER INDEX: WHERE THE INSTANCE COMING FROM.
        #                   Say if we use instance searching_history[3][7(instance 7)] to update the searching_history[4][2] (using pair(A,a),
        #                   Then searching_history[4][2][0] = 7,
        #                        searching_history[4][2][1] = A,
        #                        searching_history[4][2][2] = a

   #     GPU_SEARCH_RANGE = cuda.to_device(SEARCH_RANGE)
        GPU_one_hop_score_table = cp.array(self.one_hop_score_table)
        GPU_template_graph_adj_matrix = cp.array(template_graph.adj_matrix)
        GPU_data_graph_adj_matrix = cp.array(self.adj_matrix)
        searching_history = cp.zeros([template_graph.n + 2, SEARCH_RANGE + 1, 3], dtype=int)
        searching_value = cp.zeros([template_graph.n + 2, SEARCH_RANGE + 1], dtype=float)

        tmp_value = np.zeros(SEARCH_RANGE * ADJ_MAX * template_graph.n + 1, dtype=float)
        tmp_history = np.zeros((SEARCH_RANGE * ADJ_MAX * template_graph.n + 1, 3), dtype=int)

        #for j in range(1, template_graph.n + 1):
        j = 1
        for i in range(1, self.n + 1):
          tmp_value[(i - 1) * template_graph.n + j - 1] = self.one_hop_score_table[i][j]
          tmp_history[(i - 1) * template_graph.n + j - 1] = (0, i, j)
        tmp_value= cp.array(tmp_value, dtype = float)
        tmp_history = cp.array(tmp_history, dtype = int)
        #print(type(tmp_value))
        #print(type(tmp_history))

        self.GPU_search_update_results(1, tmp_value, tmp_history, searching_history, searching_value)

        for current_lvl in range(1, template_graph.n):
          # self.hashing_table = set()
          GPU__trust_searching_matrix_type[SEARCH_RANGE,1](self.n, template_graph.n, current_lvl, GPU_one_hop_score_table, GPU_template_graph_adj_matrix, GPU_data_graph_adj_matrix, searching_history, searching_value, tmp_history, tmp_value)
          #DATA_NODE_SIZE : int , TEMPLATE_NODE_SIZE : int ,current_lvl, GPU_one_hop_score_table, GPU_template_graph_adj_matrix, GPU_data_graph_adj_matrix,searching_history, searching_value, new_value, new_history
         # print(cp.max(tmp_value))
          self.GPU_search_update_results(current_lvl + 1, tmp_value, tmp_history, searching_history, searching_value)
         # print(searching_history[current_lvl])
         # print(searching_value[current_lvl])
          
        self.GPU_TRUST_output_result_matrix(template_graph.n, searching_value, searching_history)

        
    def GPU_TRUST_output_result_matrix(self, cur_level, searching_value, searching_history):
        n = cur_level
        print("PRINTING TOP K BEST SOLUTION....")
        cnt = 0
        return
        for searching_index in range(0, SEARCH_RANGE):
            fa, x, y = searching_history[n][searching_index]
            if x != 0 and y != 0:
                cnt += 1
                print("result No.", cnt)
                print("---Avg One-hop-value", searching_value[n][searching_index] / n)
                print("---Matching info(data node, template node):")
                if DETAIL_RESULT == True:
                    for k in range(n - 1, 0, -1):
                        print(x, y)
                        fa, x, y = searching_history[k][fa]
                    print(x, y)
                #return

    def output_adj_matrix(self):
        print(self.n)
        for i in range(1, self.n + 1):
          print(self.adj_matrix[i][1 : self.adj_matrix[i][0] + 1])

#build a graph
def create_graph(size : int, edge_size : int) -> Graph:
    #create node
    node_list : List[Node] = []
    node_size = size
    for i in range(1, size + 1):
        name = random.random()
        new_node = Node(str(name), random.randint(0, 100))
        node_list += [new_node]
    #create edges
    edge_list : List[Edge] = []
    t = np.zeros((node_size, ADJ_MAX))
    adj_num = np.zeros(node_size, dtype = int)
    for j in range(1, edge_size + 1):
        name = random.random()
        p = q = 0
        flag = True
        while flag:
          p = random.randint(0, size - 1)
          q = random.randint(0, size - 1)
          if (p == q or adj_num[p] >= ADJ_MAX or adj_num[q] >= ADJ_MAX):
            continue
          flag = False
          for i in range(0, adj_num[p]):
            if t[p][i] == q:
              flag = True
              break
          for i in range(0, adj_num[q]):
            if t[q][i] == p:
              flag = True
              break
        t[p][adj_num[p]] = q
        t[q][adj_num[q]] = p
        adj_num[p] += 1
        adj_num[q] += 1
        new_edge = Edge(str(name), node_list[p].name, node_list[q].name, random.randint(0, 100))
        edge_list += [new_edge]
    return Graph(node_size, edge_size, node_list, edge_list)

# variables to vary
A = create_graph(10000, 40000)
B = create_graph(3, 3)
# change them according to excel table 

print("Building the similarity score")
time_0 = time.time()
A.build_word_distance(B)
time_A = time.time()
print("SIMILARITY RUNNING TIME:", time_A - time_0)

print("Building the one hop score...Speeding_up(GPU)")
A.GPU_build_node_based_one_hop_score(B)
time_B = time.time()
print("Building Complete.")
print("ONE_HOP_SCORE RUNNING TIME:", time_B - time_A)

print("Building the one hop score...Speeding_up(CPU)")
#A.CPU_build_node_based_one_hop_score(B)
A.OLD_build_node_based_one_hop_score(B)
time_C = time.time()
print("Building Complete.")
print("ONE_HOP_SCORE RUNNING TIME:", time_C - time_B)

print("Building the one hop score...(OLD)")
#A.GPU_build_node_based_one_hop_score(B)
#A.OLD_build_node_based_one_hop_score(B)
print("TOO SLOW SKIPPED")
time_D = time.time()

print("Building Complete.")
print("ONE_HOP_SCORE RUNNING TIME:", time_D - time_C)
#print adj matrix and one-hop score matrixa
#print("adj-matrix of data graph", A.output_adj_matrix())
#print("adj-matrix of template graph", B.output_adj_matrix())
print("One Hop Score Matrix:\n", A.one_hop_score_table[1:A.n + 1, 1:B.n + 1])
#print("One Hop Score Matrix New:\n", A.one_hop_score_table_CPU[1:A.n + 1, 1:B.n + 1])

print("Building the Trust Searching Tree...Using...GPU....")
# A.search_best_fitting_graph(B)
#A.search_best_fitting_graph_matrix_type(B)
A.GPU_search_best_fitting_graph_matrix_type(B)
time_E = time.time()
print("TRUST RUNNING TIME:", time_E - time_D)






Found 1 CUDA devices
id 0            b'Tesla K80'                              [SUPPORTED]
                      compute capability: 3.7
                           pci device id: 4
                              pci bus id: 0
Summary:
	1/1 devices are supported
Building the similarity score
SIMILARITY RUNNING TIME: 48.969730615615845
Building the one hop score...Speeding_up(GPU)
Building Complete.
ONE_HOP_SCORE RUNNING TIME: 1.241767168045044
Building the one hop score...Speeding_up(CPU)
Building Complete.
ONE_HOP_SCORE RUNNING TIME: 864.107602596283
Building the one hop score...(OLD)
TOO SLOW SKIPPED
Building Complete.
ONE_HOP_SCORE RUNNING TIME: 0.0022552013397216797
One Hop Score Matrix:
 [[0.44722222 0.44722222 0.47222222]
 [0.4875731  0.54586988 0.46253655]
 [0.4494152  0.45263158 0.4494152 ]
 ...
 [0.51023392 0.47888239 0.50730994]
 [0.48099415 0.50268031 0.45004873]
 [0.65935673 0.6505848  0.61354776]]
Building the Trust Searching Tree...Using...GPU....
PRINTING TOP K BEST SOLUTI

In [ ]:
!lscpu |grep 'Model name'

Model name:          Intel(R) Xeon(R) CPU @ 2.30GHz


In [ ]:
#Below is I alter the normal "recurrsive" max-matching algorithm to a non-recurrsive one.
#I consider this edition pretty fun


def max_matching_algo(w : np.array, n: int) -> float:
 #   print("matching....")
    lx = np.zeros([n + 2], dtype = float)
    ly = np.zeros([n + 2], dtype = float)
    visited_x = np.zeros([n + 2], dtype=int)
    visited_y = np.zeros([n + 2], dtype=int)
    link_y = np.zeros([n + 2], dtype=int)
    global lack

    def find(x: int) -> bool:
        global lack
        visited_x[x] = True
        for y in range(1, n + 1):
            if not visited_y[y]:
                tmp = lx[x] + ly[y] - w[x][y]
                if tmp < eps:
                    visited_y[y] = True
                    if link_y[y] == 0 or find(link_y[y]):
                        link_y[y] = x
                        return True
                else:
                    lack = min(lack, tmp)
        return False
    for i in range(1, n + 1):
        for j in range(1, n + 1):
            lx[i] = max(lx[i], w[i][j]) # init

    for x in range(1, n + 1):
        while True:
            visited_x = np.zeros([n + 2], dtype=int)
            visited_y = np.zeros([n + 2], dtype=int)
            lack = 100000
            if find(x): break
            for i in range(1, n + 1):
                if visited_x[i]:
                    lx[i] -= lack
                if visited_y[i]:
                    ly[i] += lack
    total_value = 0.

    for y in range(1, n + 1):
        total_value += w[link_y[y]][y]

  #  print("matching Complete!")
    return total_value



def GPU_max_matching_algo(w : np.array, n: int) -> float:
 #   print("matching....")
    lx = np.zeros([n + 2], dtype = float)
    ly = np.zeros([n + 2], dtype = float)
    visited_x = np.zeros([n + 2], dtype=int)
    visited_y = np.zeros([n + 2], dtype=int)
    link_y = np.zeros([n + 2], dtype=int)

    for i in range(1, n + 1):
        for j in range(1, n + 1):
            lx[i] = max(lx[i], w[i][j]) # init

    for x in range(1, n + 1):
      while True:
        lack_GPU = 100000
        visited_x = np.zeros([n + 2], dtype=int)
        visited_y = np.zeros([n + 2], dtype=int)

        pos = 1
        stack_x = np.zeros(n, dtype = int)
        stack_y = np.zeros(n, dtype = int)
        stack_x[1] = x
        stack_y[1] = 0
        Label = False
        while pos > 0:
          cur_x = stack_x[pos]
          cur_y = stack_y[pos]
          Recur_label = False
          if Label:
            link_y[cur_y] = cur_x
            pos -= 1
            continue
          visited_x[cur_x] = True
          for y in range(cur_y + 1, n + 1):
            if not visited_y[y]:
              tmp_GPU = lx[cur_x] + ly[y] - w[cur_x][y]
              if tmp_GPU < eps:
                  visited_y[y] = True
                  if link_y[y] == 0:
                    link_y[y] = cur_x
                    pos -=1
                    Label = True
                  else:
                    stack_y[pos] = y
                    pos += 1
                    stack_x[pos] = link_y[y]
                    stack_y[pos] = 0
                    Recur_label = True
                  break
              else:
                  lack_GPU = min(lack_GPU, tmp_GPU)
          if Recur_label == False and Label == False: pos -= 1

        if Label: break

        for i in range(1, n + 1):
            if visited_x[i]:
                lx[i] -= lack_GPU
            if visited_y[i]:
                ly[i] += lack_GPU
    total_value = 0.

    for y in range(1, n + 1):
        total_value += w[link_y[y]][y]

  #  print("matching Complete!")
    return total_value

n = 105
p = np.random.rand(n + 1, n + 1)
output_B = GPU_max_matching_algo(p,n)
output_A = max_matching_algo(p,n)
output_C = GPU_max_matching_algo(p,n)
print(output_A, output_B, output_C)

[1 2 3 4 5] [1 2 3 4 5] [1 2 3 4 5]
103.41049129599283 103.41049129599283 103.41049129599283
